# get ann

In [1]:
from cmschina_tianyan.session import CmsSession
import pandas as pd
dataset = CmsSession.init(appId="cb632213cf764a019b32f8cc9d2040b7", appSecret="b23700b9fbf0090709c7b72ac0fdcb390cd19cc663bcf1c7a07c12b8c32bf377")

stk_code_list = ['300587.SZ',
# '603185.SH', #转债已退市
'600674.SH',
'002203.SZ',
'601127.SH',
'002597.SZ',
'603348.SH',
# '300487.SZ', #转债已退市
'603876.SH',
'601677.SH',
'300655.SZ',
'002078.SZ',
'002738.SZ', #2022-10-14新增4个对子
'002812.SZ',
'603596.SH',
'300332.SZ',#2022-11-18新增3个对子
'300586.SZ',
'300382.SZ',
'002840.SZ'
]

# stk_code = '603355.SH'
df_ann_res = []
for stk_code in stk_code_list:
    sql = f''' 
    SELECT * FROM wind_admin.ASHAREANNINF 
    where S_INFO_WINDCODE='{stk_code}' 
    order by ANN_DT desc
    '''

    df_ann = dataset.get_data(sql)
    df_ann_res.append(df_ann)
    # df_ann.to_excel(f'./ann/{stk_code}.xlsx')
df_total_ann = pd.concat(df_ann_res)
df_total_ann['serial'] = [i for i in range(len(df_total_ann))]
df_total_ann['author'] = ['上市公司']*len(df_total_ann)
df_total_ann = df_total_ann.set_index('serial')
df_total_ann = df_total_ann[['S_INFO_WINDCODE','ANN_DT','N_INFO_TITLE','author']]
cols = ['code','date','title','author']
df_total_ann.columns = cols
df_total_ann    

[2022-12-13T13:14:24.526693+0800] [12598] [INFO] [session.py] [63] PROD Env Init Completed!
[2022-12-13T13:14:25.539786+0800] [12598] [DEBUG] [tianyan_provider.py] [130] retrieve total 1291 rows of data
[2022-12-13T13:14:27.306921+0800] [12598] [DEBUG] [tianyan_provider.py] [130] retrieve total 2383 rows of data
[2022-12-13T13:14:27.682700+0800] [12598] [DEBUG] [tianyan_provider.py] [130] retrieve total 2121 rows of data
[2022-12-13T13:14:28.049941+0800] [12598] [DEBUG] [tianyan_provider.py] [130] retrieve total 1586 rows of data
[2022-12-13T13:14:28.337500+0800] [12598] [DEBUG] [tianyan_provider.py] [130] retrieve total 1585 rows of data
[2022-12-13T13:14:28.567110+0800] [12598] [DEBUG] [tianyan_provider.py] [130] retrieve total 742 rows of data
[2022-12-13T13:14:28.791903+0800] [12598] [DEBUG] [tianyan_provider.py] [130] retrieve total 834 rows of data
[2022-12-13T13:14:29.075269+0800] [12598] [DEBUG] [tianyan_provider.py] [130] retrieve total 1507 rows of data
[2022-12-13T13:14:30.3

,code,date,title,author
serial,,,,
0,300587.SZ,2022-12-09,天铁股份:浙江天铁实业股份有限公司关于子公司使用部分闲置自有资金购买理财产品的公告,上市公司
1,300587.SZ,2022-12-07,天铁股份:浙江天铁实业股份有限公司关于延期回复深圳证券交易所关于公司申请向特定对象发行股票的...,上市公司
2,300587.SZ,2022-11-30,天铁股份:浙江天铁实业股份有限公司关于部分监事股份减持的进展公告,上市公司
3,300587.SZ,2022-11-24,天铁股份:中泰证券股份有限公司关于浙江天铁实业股份有限公司向特定对象发行股票之发行保荐书,上市公司
4,300587.SZ,2022-11-24,天铁股份:浙江天册律师事务所关于浙江天铁实业股份有限公司向特定对象发行A股股票的补充法律意见...,上市公司
...,...,...,...,...
25569,002840.SZ,2016-12-16,华统股份:北京市天元律师事务所关于公司首次公开发行股票并上市的补充法律意见(四),上市公司
25570,002840.SZ,2016-12-16,华统股份:首次公开发行股票招股意向书摘要,上市公司
25571,002840.SZ,2016-12-16,华统股份:关于公司最近三年及一期非经常性损益的鉴证报告,上市公司


# get news

In [2]:
import pandas as pd
import requests
import random
import time
import os
import openpyxl
from bs4 import BeautifulSoup
def get_url(code,pages):
    '''
    获取东方财富网股吧链接列表
    code是指公司代码
    page是值爬取页数
    '''
    url_list = []
    for page in range(1,pages+1):
        url = f"http://guba.eastmoney.com/list,{code},1,f_{page}.html"
        url_list.append(url)
        
    return url_list

def get_news(url_list,code):
    '''
    获取东方财富网新闻列表至本地xls
    url_list是指链接列表
    '''
    headers = {
        # 'User-Agent': UserAgent(verify_ssl=False).random,
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
        'cookie': 'qgqp_b_id=b90ae58b50ba4b8868c1988b80823e4c; st_si=62771611099798; st_asi=delete; st_pvi=50748890314615; st_sp=2022-10-24 11:22:01; st_inirUrl=http://guba.eastmoney.com/list,hk01810,1,f_1.html; st_sn=2; st_psi=20221024112234183-117001300541-9748756866'
    }
    
    # 保存爬取内容
    outwb = openpyxl.Workbook() # 打开一个将写的文件
    outws = outwb.create_sheet(index=0) # 在将写的文件创建sheet
    outws.cell(row = 1, column = 1, value = "read")
    outws.cell(row = 1, column = 2, value = "comment")
    outws.cell(row = 1, column = 3, value = "title")
    outws.cell(row = 1, column = 4, value = "author")
    outws.cell(row = 1, column = 5, value = "renew")
    outws.cell(row = 1, column = 6, value = "link")
    index = 2
    
    for i in range(len(url_list)):
        url = url_list[i]
        res = requests.get(url,headers = headers)
        res.encoding = res.apparent_encoding
        html = res.text
        soup = BeautifulSoup(html,"html.parser")
        read_list = soup.select(".l1.a1")[1:]
        comment_list = soup.select(".l2.a2")[1:]
        title_list = soup.select(".l3.a3")[1:]
        author_list = soup.select(".l4.a4")[1:]
        renew_list = soup.select(".l5.a5")[1:]
        for k in range(len(title_list)):
            outws.cell(row = index, column = 1, value = str(read_list[k].text.strip()))
            outws.cell(row = index, column = 2, value = str(comment_list[k].text.strip()))
            outws.cell(row = index, column = 3, value = str(title_list[k].select('a')[0]["title"]))
            outws.cell(row = index, column = 4, value = str(author_list[k].text.strip()))
            outws.cell(row = index, column = 5, value = str(renew_list[k].text.strip()))
            outws.cell(row = index, column = 6, value = str(title_list[k].select('a')[0]["href"]))                                                
            index += 1
            # print(str(title_list[k].select('a')[0]["href"]))
            # print(title_list[k].select('a')[0]["title"],renew_list[k].text.strip())
        time.sleep(random.uniform(3,4))
    save_path = f"./news_ann/news/tmp/{code}.xlsx"            
    outwb.save(save_path)
    df = pd.read_excel(save_path)
    return df 
 
stk_code_list = ['300587.SZ',
# '603185.SH', #转债已退市
'600674.SH',
'002203.SZ',
'601127.SH',
'002597.SZ',
'603348.SH',
# '300487.SZ', #转债已退市
'603876.SH',
'601677.SH',
'300655.SZ',
'002078.SZ',
'002738.SZ', #2022-10-14新增4个对子
'002812.SZ',
'603596.SH',
'300332.SZ',#2022-11-18新增3个对子
'300586.SZ',
'300382.SZ',
'002840.SZ'
]

df_news_res = []
for stk_code in stk_code_list:
    code = stk_code.split('.')[0]
    pages = 1
    url_list = get_url(code,pages)
    df_tmp = get_news(url_list,code)
    df_tmp['code'] = [stk_code]*len(df_tmp)
    df_news_res.append(df_tmp)
    print(f"{code} 运行完成")    
df_total_news = pd.concat(df_news_res)

300587 运行完成
600674 运行完成
002203 运行完成
601127 运行完成
002597 运行完成
603348 运行完成
603876 运行完成
601677 运行完成
300655 运行完成
002078 运行完成
002738 运行完成
002812 运行完成
603596 运行完成
300332 运行完成
300586 运行完成
300382 运行完成
002840 运行完成


In [3]:
df_total_news = pd.concat(df_news_res)
df_total_news['serial'] = [i for i in range(len(df_total_news))]
df_total_news = df_total_news.set_index('serial')

df_total_news['date'] = df_total_news['renew'].apply(lambda x:'2022-' + x.split(' ')[0])
cols = ['code','date','title','author']
df_total_news = df_total_news[cols]
df_total_news

,code,date,title,author
serial,,,,
0,300587.SZ,2022-12-13,天铁股份：连续3日融资净买入累计1958.54万元（12-12）,天铁股份资讯
1,300587.SZ,2022-12-13,天铁股份12月12日被深股通减持88.75万股,天铁股份资讯
2,300587.SZ,2022-12-12,天铁股份：融资净买入618.09万元，融资余额4.45亿元（12-09）,天铁股份资讯
3,300587.SZ,2022-12-11,天铁股份本周融资净买入1951.52万元，居橡胶制品板块第二,大眼看两融
4,300587.SZ,2022-12-11,天铁股份本周深股通持股市值减少1391.18万元，居橡胶制品板块第四,胖猫周周见
...,...,...,...,...
1355,002840.SZ,2022-10-25,华统股份：第三季净利1.04亿元 同比扭亏为盈,华统股份资讯
1356,002840.SZ,2022-10-25,华统股份：三季度净利1.04亿元 同比增200.95%,华统股份资讯
1357,002840.SZ,2022-10-25,华统股份：2022年前三季度净利润约4537万元,华统股份资讯


# merge

In [4]:
# df_total_ann = pd.DataFrame([])

In [5]:
df_total_news_ann = pd.concat([df_total_ann,df_total_news])
df_total_news_ann['serial'] = [i for i in range(len(df_total_news_ann))]
df_total_news_ann = df_total_news_ann.set_index('serial')


In [6]:
from WindPy import *
#Wind API 使用W账户密码登录
w.start() 
#return True and then can go to next operation.
w.isconnected()

22.11.1.52440
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [7]:
df_stk_name = w.wss('300587.SZ,603185.SH,600674.SH,002203.SZ,601127.SH,002597.SZ,603348.SH,300487.SZ,603876.SH,601677.SH,300655.SZ,002078.SZ,002738.SZ, 002812.SZ,603596.SH,300332.SZ,300586.SZ,300382.SZ,002840.SZ', "sec_name", "",usedf=True)[1]
stk_name_dic = df_stk_name.to_dict()['SEC_NAME']
# df_stk_name
df_total_news_ann['name'] =  df_total_news_ann['code'].map(stk_name_dic)
ordered_cols = ['code','name','date','title','author']

df_total_news_ann = df_total_news_ann [ordered_cols]
df_total_news_ann['code_name'] = df_total_news_ann['code'] + df_total_news_ann['name']
df_total_news_ann

,code,name,date,title,author,code_name
serial,,,,,,
0,300587.SZ,天铁股份,2022-12-09,天铁股份:浙江天铁实业股份有限公司关于子公司使用部分闲置自有资金购买理财产品的公告,上市公司,300587.SZ天铁股份
1,300587.SZ,天铁股份,2022-12-07,天铁股份:浙江天铁实业股份有限公司关于延期回复深圳证券交易所关于公司申请向特定对象发行股票的...,上市公司,300587.SZ天铁股份
2,300587.SZ,天铁股份,2022-11-30,天铁股份:浙江天铁实业股份有限公司关于部分监事股份减持的进展公告,上市公司,300587.SZ天铁股份
3,300587.SZ,天铁股份,2022-11-24,天铁股份:中泰证券股份有限公司关于浙江天铁实业股份有限公司向特定对象发行股票之发行保荐书,上市公司,300587.SZ天铁股份
4,300587.SZ,天铁股份,2022-11-24,天铁股份:浙江天册律师事务所关于浙江天铁实业股份有限公司向特定对象发行A股股票的补充法律意见...,上市公司,300587.SZ天铁股份
...,...,...,...,...,...,...
26929,002840.SZ,华统股份,2022-10-25,华统股份：第三季净利1.04亿元 同比扭亏为盈,华统股份资讯,002840.SZ华统股份
26930,002840.SZ,华统股份,2022-10-25,华统股份：三季度净利1.04亿元 同比增200.95%,华统股份资讯,002840.SZ华统股份
26931,002840.SZ,华统股份,2022-10-25,华统股份：2022年前三季度净利润约4537万元,华统股份资讯,002840.SZ华统股份


# output to excel

In [8]:
# import openpyxl 
# import xlwings as xw
# wb_file = './news_ann/total_news_ann/news_ann.xlsx'
# book = openpyxl.load_workbook(wb_file)   #读取你要写入的workbook
# #和pd.read_excel() 用于将Dataframe写入excel。xls用xlwt。xlsx用openpyxl
# writer = pd.ExcelWriter(wb_file, engine='openpyxl')   
# ##此时的writer里还只是读写器. 然后将上面读取的book复制给writer  
# writer.book = book
# #转化为字典的形式
# writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
# #将data写入writer
# df_total_news_ann.to_excel(writer,sheet_name="sheet1",index=False)
# writer.save()

In [9]:
df_total_news_ann.to_excel('./news_ann/total_news_ann/news_ann.xlsx')